# Best practices to work with your data

## Filter for documents that belong to one category

In Project 46 you want to get all documents that belong to category with ID 63.

In [2]:
from konfuzio_sdk.data import Project

prj = Project(id_=46)
category = prj.get_category_by_id(63)

category.documents()

[Gehalt.pdf: 44823,
 Festlohn.pdf: 44834,
 vermögenswirksame Leistungen.pdf: 44839,
 betriebliche Altersvorsorge AG finanziert.pdf: 44840,
 Weihnachtsgeld.pdf: 44841,
 Stundenlohn.pdf: 44842,
 Fahrtkostenzuschuss pauschal versteuert.pdf: 44843,
 Betirebliche Altersvorsorge Mischfinanzierung.pdf: 44845,
 Darlehen.pdf: 44846,
 Dienstwagen mit Gehaltsverzicht.pdf: 44847,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_1.pdf: 44848,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_2.pdf: 44850,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_4.pdf: 44851,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_3.pdf: 44852,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_5.pdf: 44853,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_6.pdf: 44854,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_7.pdf: 44855,
 Auswertungspaket - unterschiedliche B_N-Auswertungen.pdf_8.pdf: 44856,
 Auswertungspaket - unterschiedliche B_N-Auswertungen